In [ ]:
!pip3 install calmap
!pip3 install bar_chart_race
!pip3 install pywaffle

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import calmap
from datetime import datetime
import json
import requests
plt.style.use('fivethirtyeight')
%matplotlib inline

#bar chart race
import bar_chart_race as bcr

# Waffle Chart
from pywaffle import Waffle

# plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

#geopandas
import geopandas as gpd


# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801'

import warnings
warnings.filterwarnings("ignore")

import os

def print_files():
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

In [ ]:
print_files()

In [ ]:
covid_19_india = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

In [ ]:
covid_19_india.head()

In [ ]:
covid_19_india = covid_19_india.drop(['Sno','Time','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
covid_19_india = covid_19_india.rename(columns={'State/UnionTerritory':'States','Cured':'Recovered'})

In [ ]:
covid_19_india['Active'] = covid_19_india['Confirmed'] - covid_19_india['Recovered'] - covid_19_india['Deaths']

In [ ]:
covid_19_india = covid_19_india.sort_values(['Date', 'States']).reset_index(drop=True)
covid_19_india['Date'] = pd.to_datetime(covid_19_india['Date'])


In [ ]:
covid_19_india

In [ ]:
india_wise_cases = covid_19_india[covid_19_india['Date'] == covid_19_india['Date'].max()].copy().fillna(0)
india_wise_cases.index = india_wise_cases["States"]
india_wise_cases = india_wise_cases.drop(['States', 'Date'], axis=1)

In [ ]:
df = pd.DataFrame(pd.to_numeric(india_wise_cases.sum()),dtype=np.float64).transpose()
df.style.background_gradient(cmap='summer_r',axis=1)

In [ ]:
india_wise_cases.sort_values('Active', ascending= False).style\
    .background_gradient(cmap='BrBG_r', subset=['Confirmed'])\
    .background_gradient(cmap='YlGn_r', subset=["Deaths"])\
    .background_gradient(cmap='YlOrBr',subset=['Recovered'])\
    .background_gradient(cmap='BuPu', subset=['Active'])

In [ ]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [ ]:
top_10_confirmed_states = india_wise_cases.sort_values('Confirmed', ascending=False)[:10]
horizontal_bar_chart(top_10_confirmed_states,top_10_confirmed_states.Confirmed, top_10_confirmed_states.index,
                     'Top 10 States with most confirmed cases', 'Number of Confirmed cases (in Thousands)','State Name',cnf)

In [ ]:
top_10_death_states = india_wise_cases.sort_values('Deaths',ascending = False)[:10]
horizontal_bar_chart(top_10_death_states,top_10_death_states.Deaths, top_10_death_states.index, 'Top 10 States with most deaths','Number of deaths(In Thousands)','State Name',dth)

In [ ]:
last_date = covid_19_india['Date'].max()
print(last_date)

In [ ]:
df_last_date = covid_19_india[covid_19_india['Date'] == last_date]
series_last_date = df_last_date.groupby('States')['Confirmed'].sum().sort_values(ascending=False)

labels = []
values = []
state_count = 5
other_total = 0
for state in series_last_date.index:
    if state_count > 0:
        labels.append(state)
        values.append(series_last_date[state])
        state_count -= 1
    else:
        other_total += series_last_date[state]
labels.append("Other")
values.append(other_total)

wedge_dict = {
    'edgecolor': 'black',
    'linewidth': 1        
}

explode = (0.2, 0, 0, 0, 0, 0)
fig = plt.figure(figsize=(15,9))
plt.title(f"Total Cases on {last_date}")
plt.pie(values, labels=labels, explode=explode, autopct='%1.1f%%', wedgeprops=wedge_dict)
plt.show()

In [ ]:
fp = '../input/india-2020-gisdata/Igismap/Indian_States.shp'
map_df = gpd.read_file(fp)
# change state name to match in both files -- manually checked few
map_df['st_nm'].iloc[0]  = 'Andaman and Nicobar Islands'
map_df['st_nm'].iloc[12] = 'Jammu and Kashmir'
map_df['st_nm'].iloc[6] = 'Dadara and Nagar Havelli'
map_df['st_nm'].iloc[23] = 'Delhi'

In [ ]:
merged = map_df.set_index('st_nm').join(covid_19_india.set_index('States'))

#fill NaN values with Zero

merged[['Confirmed', 'Recovered', 'Deaths']] = merged[['Confirmed', 'Recovered', 'Deaths']].fillna(0).astype('int')

In [ ]:
fig, ax = plt.subplots(1, figsize=(23, 19))
ax.axis('on')
ax.set_title('Total COVID-19 cases in India', fontdict={'fontsize': '35', 'fontweight' : '5'})

# plot the figure
merged.plot(column='Confirmed', cmap='Oranges', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

In [ ]:
fig, ax = plt.subplots(1, figsize=(23, 19))
ax.axis('on')
ax.set_title('Total Deaths due to COVID-19 in India', fontdict={'fontsize': '35', 'fontweight' : '5'})

# plot the figure
merged.plot(column='Deaths', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

### **Bar chart Race**

In [ ]:
bcr_deaths = covid_19_india.pivot_table('Deaths',['Date'], 'States')
bcr_deaths.sort_values(list(bcr_deaths.columns),inplace=True)
bcr_deaths = bcr_deaths.sort_index()

In [ ]:
bcr_deaths.head()

In [ ]:
bcr.bar_chart_race(
    df=bcr_deaths,
    filename=None,
    #filename='Deaths cases Bar Chart Race.mp4',
    orientation='h',
    sort='desc',
    n_bars=10,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=10,
    interpolate_period=False,
    label_bars=True,
    bar_size=.90,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                      's': f'Total Deaths Cases: {v.nlargest(10).sum():,.0f}',
                                      'ha': 'right', 'size': 8, 'family': 'Courier New'},
    period_length=500,
    figsize=(6,5),
    dpi=144,
    cmap='dark12',
    title='Deaths due to Corona Virus by State.',
    title_size='',
    bar_label_size=7,
    tick_label_size=5,
    shared_fontdict={'family' : 'Helvetica','color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=True)